In [1]:
#Import Modules
import act

import pandas as pd
import numpy as np

from datetime import datetime
from datetime import timedelta

In [40]:
#Create a timewindow for which to look through
time_window = [datetime(2001, 1, 1, 0, 0), datetime(2001, 4, 1, 0, 0)]

#Create a list of ASOS stations to look through
#stations = ("ORD", "JOT", "PWK", "UGN", "DPA", "ARR", "LOT", "IGQ")
#stations = ("ORD", "LOT", "PWK", "UGN", "DPA", "ARR", "IGQ")
stations = ("ORD", "LOT", "UGN", "DPA", "ARR")
#stations = ("DPA", "LOT")

In [41]:
now = datetime.now()
print(now)

2021-01-22 13:13:03.423035


In [42]:
#Create arrays that will allow hold all the data for later conversion
array = []
two = []
#Create for-loop to go throught list of stations.
for i in stations:
    my_asoses = act.discovery.get_asos(time_window, station = i)
    
    #Extract rain totals from ASOS station
    station = my_asoses[i]
    rain = station.p01i.values
    lat = station.lat.values
    lon = station.lon.values
    
    #Extract datetimes and create a better format for output time
    time = station.time.values
    time2 = pd.to_datetime(time)
    date = time2.strftime('%d %B %Y')
    hour = time2.strftime('%X')
    
    r = 0
    total = 0
    
    for j in range(len(rain)):
        
        if rain[j] >= 0.001:
            
            if r == 0:
                array = []
                array.append((i, hour[j], date[j]))
                array.append((lat[j], lon[j]))
                current = time2.hour[j]
                start_rain = datetime.strptime((date[j] +' '+ hour[j]), '%d %B %Y %X')
                r = rain[j]
                
            elif r > 0 and current == time2.hour[j]:
                if rain[j] >= r:
                    r = rain[j]
                
            elif r > 0 and current != time2.hour[j]:
                current = time2.hour[j]
                total = total + r
                r = 0.001
        
        
        elif rain[j] < 0.001 and r > 0:
            end_rain = datetime.strptime((date[j] +' '+ hour[j]), '%d %B %Y %X')
            r = total + r
            total_time = end_rain - start_rain
            rainfall_per_hour = r/total_time.total_seconds()*3600
            array.append((hour[j], date[j], r, rainfall_per_hour))
            two.append(array)
            r = 0
            total = 0

Downloading: ORD
Downloading: LOT
Downloading: UGN
Downloading: DPA
Downloading: ARR


In [43]:
#Create arrays for values from above to better organized into a pandas dataframe.
station = []
start_time = []
start_date = []
end_time = []
end_date = []
rain_total = []
rainfall = []
lat = []
lon = []

#Append values into separate arrays
for i in range(len(two)):
    station.append(two[i][0][0])
    start_time.append(two[i][0][1])
    start_date.append(two[i][0][2])
    lat.append(two[i][1][0])
    lon.append(two[i][1][1])
    end_time.append(two[i][2][0])
    end_date.append(two[i][2][1])
    rain_total.append(two[i][2][2])
    rainfall.append(two[i][2][3])
    
station = np.array(station)
start_time = np.array(start_time)
start_date = np.array(start_date)
end_time = np.array(end_time)
end_date = np.array(end_date)
rain_total = np.array(rain_total)
rainfall = np.array(rainfall)

#Create dictionary for easier conversion to a pandas dataframe
f = {'station': station,
     'start_time': start_time,
     'start_date': start_date,
     'end_time': end_time,
     'end_date': end_date,
     'rain_total': rain_total,
     'rainfall_per_hour': rainfall,
     'Lat': lat,
     'Lon': lon}

In [44]:
#Create the pandas dataframe
df = pd.DataFrame(f)
df

,station,start_time,start_date,end_time,end_date,rain_total,rainfall_per_hour,Lat,Lon
0,ORD,23:56:00,29 January 2001,17:56:00,30 January 2001,0.030,0.001667,41.9875,-87.9319
1,ORD,22:25:00,08 February 2001,23:56:00,08 February 2001,0.010,0.006593,41.9875,-87.9319
2,ORD,11:56:00,09 February 2001,12:30:00,25 February 2001,0.041,0.000107,41.9875,-87.9319
3,ORD,17:56:00,12 March 2001,23:56:00,14 March 2001,0.080,0.001481,41.9875,-87.9319
4,ORD,05:56:00,15 March 2001,12:45:00,15 March 2001,0.011,0.001614,41.9875,-87.9319
...,...,...,...,...,...,...,...,...,...
136,ARR,04:36:00,16 March 2001,05:53:00,16 March 2001,0.010,0.007792,41.7700,-88.4800
137,ARR,23:53:00,16 March 2001,02:45:00,17 March 2001,0.012,0.004186,41.7700,-88.4800
138,ARR,03:36:00,17 March 2001,04:53:00,17 March 2001,0.010,0.007792,41.7700,-88.4800
139,ARR,05:53:00,17 March 2001,06:40:00,17 March 2001,0.010,0.012766,41.7700,-88.4800


In [45]:
#Subset values from the pandas dataframe for use in logic statements
station = df.station.values
start_time = df.start_time.values
start_date = df.start_date.values
end_time = df.end_time.values
end_date = df.end_date.values
rain_total = df.rain_total.values
index = df.index.values
rainfall = df.rainfall_per_hour

In [46]:
dupe = []
name_dupe = []

for k in index:
    
    if rain_total[k] >= 1.5:
        datetime_start = datetime.strptime(start_date[k], '%d %B %Y')
        datetime_end = datetime.strptime(end_date[k], '%d %B %Y')
        Event_Datasets_index = []
        
        if (datetime_start not in dupe) or (datetime_end not in dupe):
            dupe.append(datetime_start)
            dupe.append(datetime_end)
            print(start_date[k], dupe)
            
            for j in index:
                datetime_start2 = datetime.strptime((start_date[j] +' '+ start_time[j]), '%d %B %Y %X')
                datetime_end2 = datetime.strptime((end_date[j] +' '+ end_time[j]), '%d %B %Y %X')
                
                if start_date[j] == start_date[k]:
                    Event_Datasets_index.append(j)
                    
                    
        Event_Datasets = df.loc[Event_Datasets_index]
        if len(Event_Datasets) >= 1:
            c = {'station': Event_Datasets.station,
                 'start_time': Event_Datasets.start_time,
                 'start_date': Event_Datasets.start_date,
                 'end_time': Event_Datasets.end_time,
                 'end_date': Event_Datasets.end_date,
                 'rain_total': Event_Datasets.rain_total,
                 'rainfall_per_hour': Event_Datasets.rainfall_per_hour,
                 'Lat': Event_Datasets.Lat,
                 'Lon': Event_Datasets.Lon}
            csv = pd.DataFrame(c)
            name = csv.start_date.values[0]
            
            name2 = datetime.strptime(name, '%d %B %Y')
            day = name2.strftime('%d')
            month = name2.strftime('%B')
            year = name2.strftime('%Y')
            
            
            if name in name_dupe:
                csv.to_csv(f'C:/Users/jerem/OneDrive/Desktop/SAVEUR/SAVEUR Internship/Data/Rainfall_Events/{day}_{month}_{year}_Rainevent.csv', mode='a', header = False)
                print('repeat')
                print(csv)
            
            else:
                csv.to_csv(f'C:/Users/jerem/OneDrive/Desktop/SAVEUR/SAVEUR Internship/Data/Rainfall_Events/{day}_{month}_{year}_Rainevent.csv')
                name_dupe.append(name)
                print(csv)
                

In [47]:
later = datetime.now()
print(later - now)

0:06:13.605550
